In [ ]:
import os
from utils import *
import optuna

In [ ]:
data, usermap, itemmap, users = load_data2()
data_train, data_test, data_val = split_data2(data, 0, 0.2)

In [ ]:
study_name = "P3Beta"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

def objective(trial):
    topK = trial.suggest_int('topK', 1, 1000)
    alpha = trial.suggest_float('alpha', 0, 2)
    beta = trial.suggest_float('beta', 0, 2)
    implicit = trial.suggest_categorical('implicit', [True, False])
    normalize_similarity = trial.suggest_categorical('normalize_similarity', [True, False])
    
    recommender = RP3betaRecommender(data_train, verbose=False)
    recommender.fit(
        topK=topK,
        alpha=alpha,
        beta=beta,
        implicit=implicit,
        normalize_similarity=normalize_similarity,
    )
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=150)

In [ ]:
study.best_params

In [ ]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

recommender = RP3betaRecommender(data_train, verbose=False)
recommender.fit(**study.best_params)

In [ ]:
pr, re, ev_map, ev_ndcg, ev_auc = evaluator(recommender, data_train, data_val)
print("Precision: {:.8f}".format(pr))
print("Recall: {:.8f}".format(re))
print("MAP: {:.8f}".format(ev_map))

In [ ]:
submission2(recommender, users, usermap, itemmap, data_train)